In [1]:
!pip install transformers
!pip install gdown

In [2]:
import gdown
url = 'https://drive.google.com/uc?id=11ZZ5iO9HqahU_T3JP5DtPZi0uJI6eQEj'
output = 'kogpt_comment_reply_model.pt' #pt: pytorch

In [3]:
output = gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=11ZZ5iO9HqahU_T3JP5DtPZi0uJI6eQEj
From (redirected): https://drive.google.com/uc?id=11ZZ5iO9HqahU_T3JP5DtPZi0uJI6eQEj&confirm=t&uuid=5c9209c5-b56a-44ec-8828-3ea9afd9ed51
To: /content/kogpt_comment_reply_model.pt
100%|██████████| 501M/501M [00:03<00:00, 131MB/s]


In [4]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel, PreTrainedTokenizerFast

# 대화에서 사용할 특별 토큰들을 정의합니다.
Q_TKN = "<usr>" # 사용자 입력을 나타내는 토큰
A_TKN = "<sys>" # 시스템의 응답을 나타내는 토큰
BOS = '</s>'    # 문장의 시작을 나타내는 토큰
EOS = '</s>'    # 문장의 종료를 나타내는 토큰
MASK = '<unused0>'  # 마스크 토큰
SENT = '<unused1>'  # 문장 분류를 위한 토큰
PAD = '<pad>'   # 패딩 토큰

# 사전 훈련된 토크나이저 load >> text 모델이 처리할 수 있는 형태로 변환

koGPT2_Tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
                                        bos_token=BOS, eos_token=EOS, unk_token='<unk>',
                                        pad_token=PAD, mask_token=MASK)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
# 사전 훈련된 모델 환경설정(구성)
config = GPT2Config.from_pretrained('skt/kogpt2-base-v2')

In [6]:
#사전 훈련된 모델 로드
model = GPT2LMHeadModel(config)

In [7]:
# 사전 훈련된 모델의 가중치 로드 >> CPU 사용하여 로드

model.load_state_dict(torch.load('kogpt_comment_reply_model.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [34]:
# 사전훈련이 된 모델 사용하기 때문에

# 모델 평가 모드 설정
# >> 모델이 학습중이 아닐 때 동작
model.eval()

sent = '0'
# 문장 분류 위한 토큰 초기화

# 모델 사용, 답변 생성하는 함수 생성
# >> 주어진 입력 텍스트 기반, 최대 길이(max_length) 응답 생성

# 0 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "" > model > response (감)
# 1 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감" > model > response (사)
# 2 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사" > model > response (합)
# 3 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합" > model > response (니)
# 4 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합니" > model > response (다)
# 5 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합니다" > model > response (.)
# 6 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합니다." > model > response (EOS)
# Q_TKN (질문 토큰), SENT(문장토큰), A_TKN(응답토큰)

def dl_model(input_text, max_length=64):
    response = "" #응답 초기화

    for _ in range(max_length):
        input_ids = torch.LongTensor(koGPT2_Tokenizer.encode(Q_TKN + input_text + SENT + sent + A_TKN + response)).unsqueeze(dim=0)
        # unsqueeze(dim=0) : 입력값을 배치(batch)차원 더해 확장

        attention_mask = input_ids != koGPT2_Tokenizer.pad_token_id
        #attention_mask : 입력값 중 어느 부분이 padding인지 나타내는 마스크 생성
        #>> model이 padding 부분을 무시하고 실제 데이터가 있는 부분만 집중(attention)

        #  모델 예측
        pred = model(input_ids = input_ids, attention_mask= attention_mask)
        pred = pred.logits #예측결과의 로짓(logits)값

        #예측결과 중 가장 확률이 높은 토큰 선택
        gen = koGPT2_Tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
        # torch.argmax(pred, dim=-1) >> 각 위치에서 가장 높은 확률 가진 토큰의 index 선택
        #[-1] : 마지막 토큰을 gen 변수에 저장

        if gen == EOS:
            break

        # 생성된 토큰 >> 응답에 추가
        # 토큰 시작부분에 있는 '_' >> 실제로 '공백'으로 대체
        response += gen.replace('▁', ' ')

    return response.strip() #양쪽 공백 제거

In [9]:
import urllib.request
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

def generate_reply(input_text):
    output = dl_model(input_text)
    return output

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarrr_sample_submit.txt",
    filename="tarrr_sample_submit.txt",
)


('tarrr_sample_submit.txt', <http.client.HTTPMessage at 0x7f7768b46320>)

In [10]:
pd.read_csv('/content/tarrr_sample_submit.txt', delimiter='\t')

,id,comment,reply
0,1,완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌,의견 감사합니다.
1,2,맛있긴 한데 양이 너무 적어서 좀... ㅠ,의견 감사합니다.
2,3,완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨,의견 감사합니다.
3,4,한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.,의견 감사합니다.
4,5,서빙하는 분이 좀 불친절해서 기분이 좀 그랬어요.,의견 감사합니다.
...,...,...,...
95,96,가성비 최고! 이 가격에 이런 맛은 정말 만족스러워요.,의견 감사합니다.
96,97,와... 여기김치... 말도안됨... ㅁㅊ...,의견 감사합니다.
97,98,주문한 지 40분 넘게 기다려서 음식 나왔네요...,의견 감사합니다.
98,99,"아이들이랑 왔는데, 키즈 메뉴도 생각보다 맛있었어요!",의견 감사합니다.


In [11]:
df = pd.read_csv('/content/tarrr_sample_submit.txt', delimiter='\t')
df.head()

,id,comment,reply
0,1,완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌,의견 감사합니다.
1,2,맛있긴 한데 양이 너무 적어서 좀... ㅠ,의견 감사합니다.
2,3,완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨,의견 감사합니다.
3,4,한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.,의견 감사합니다.
4,5,서빙하는 분이 좀 불친절해서 기분이 좀 그랬어요.,의견 감사합니다.


In [14]:
for idx, row in df.iterrows():
    comment = row['comment']
    reply = generate_reply(comment)
    reply = reply.replace('▁', ' ')


    print(f'[{idx + 1}]')
    print('comment: ',comment)
    print('reply: ', reply)
    print('--'*50)

[1]
comment:  완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌
reply:   완전 내 스타일이에요! 고객님의 의견을 소중히 여기며, 가격과 위치 모두 만족시킬 수 있도록 노력하겠습니다. 다음에도 저희 가게를 찾아주셔서 감사합니다. 다음에도 저희 가게를 찾아주셔서 더 나은 경험을 드리도록 하겠습니다.
----------------------------------------------------------------------------------------------------
[2]
comment:  맛있긴 한데 양이 너무 적어서 좀... ㅠ
reply:   양에 대한 의견 감사합니다. 양에 대한 의견 감사합니다. 더 많은 양을 준비하도록 하겠습니다.
----------------------------------------------------------------------------------------------------
[3]
comment:  완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨
reply:   완전 내 스타일이에요! 매장 분위기와 함께 좋은 시간 보내셨다니 기쁩니다! 앞으로도 편안하고 즐거운 매장 환경을 제공하기 위해 노력하겠습니다.
----------------------------------------------------------------------------------------------------
[4]
comment:  한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.
reply:   한국의 전통 음식을 잘 표현해 냈다니 기쁩니다. 감사합니다!
----------------------------------------------------------------------------------------------------
[5]
comment:  서빙하는 분이 좀 불친절해서 기분이 좀 그랬어요.
reply:   서비스에 불만족하셨다니 진심으로 사과드립니다. 직원 교육을 

In [17]:
# 챗봇 만들기

# user message >> ai message 전달 >> 응답 생성
# 대화내역에 저장하여 응답 반환

def chat_with_user(user_message, chat_history):
    ai_message = dl_model(user_message)
    chat_history.append({'user' : user_message, 'ai' : ai_message})
    return ai_message

chat_history = []
# chat_history : user 와 ai 간 대화내역 저장 리스트

while True:
    user_message = input('user > ')
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f"AI > ", ai_message)
    print('--'*50)

user > 피자가 맛있어요
AI >  피자에 대한 칭찬 감사합니다! 피자에 대한 칭찬 감사합니다! 피자에 대한 칭찬 감사합니다! 피자에 대한 칭찬 감사합니다!
----------------------------------------------------------------------------------------------------
user > 피자가 정말 맛이 없네요
AI >  피자에 대한 피드백 감사합니다. 조리 방법을 개선하여 더 맛있는 피자를 제공하겠습니다.
----------------------------------------------------------------------------------------------------
user > quit


In [18]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.5 

In [23]:
import gradio as gr

def chat_with_user(user_message, chat_history):
    ai_message = dl_model(user_message)
    chat_history.append({'user' : user_message, 'ai' : ai_message})
    return ai_message, chat_history

#gradio interface 설정 함수
def chat_interface(user_message, chat_history):
    ai_message, updated_chat_history = chat_with_user(user_message, chat_history) #user와 ai 대화내용 담김
    return ai_message, updated_chat_history

#초기 대화 내역 설정
init_chat_history = []

#gradio 입력 출력
user_input = gr.Textbox(lines=1, label='user') #사용자 입력 텍스트
chat_output = gr.Textbox(label='AI') #AI 출력 텍스트
chat_history_state = gr.State(init_chat_history) #초기 대화 내역

# gradio 인터페이스
iface = gr.Interface(
    fn = chat_interface,
    inputs = [user_input, chat_history_state],
    outputs = [chat_output, chat_history_state],
    live = False, # 실시간 업데이터 활성화
    title = 'AI 자동 응답 생성 시스템',
    description = '사용자 의견에 대한 피드백을 AI가 자동응답해 드려요.'
)
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://86e45a0630d928237e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [30]:
#멀티턴 (화자 A, 화자 B 지속 대화하는 방식)

import gradio as gr

def chat_interface(message, chat_history):
    ai_msg = dl_model(message, chat_history)
    chat_history.append((message, ai_msg)) # 튜플형식으로 대화 기록
    return chat_history


#초기 대화 내역 설정
init_chat_history = []

iface = gr.ChatInterface(
    fn = chat_interface,
    chatbot = gr.Chatbot(), #chatbot component
    title = 'AI 자동 응답 생성 챗봇-멀티 응답',
    description = '사용자 의견에 대한 피드백을 AI가 자동응답해 드려요.'
)
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1c7ae7797c7dab7d88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
